In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_2020= pd.read_excel('2020.xlsx')
df_2021= pd.read_excel('2021.xlsx')
df_2022= pd.read_excel('2022.xlsx')

In [ ]:
df_2022.columns

In [ ]:
df_2022.info()

In [ ]:
df_2020_ingexp = df_2020[['Código NIF','Ingresos de explotación\nmil EUR\n2020']]
df_2020_ingexp.replace('n.d.',np.nan,inplace=True)
df_2020_ingexp.replace('Financial Export limit reached',np.nan,inplace=True)
df_2020_ingexp.isna().sum()


In [ ]:
df_2022_col=df_2022[['Nombre', 'Código NIF', 'Localidad',
       'Provincia', 'País', 'Código primario CNAE 2009',
       'Literal código CNAE 2009 primario', 'Fecha constitución',
       'Importador / Exportador', 'Países importación', 'Países exportación','Resultado del Ejercicio\nmil EUR\n2022',
       'Total activo\nmil EUR\n2022', 'Ingresos de explotación\nmil EUR\n2022', 'Activo circulante\nmil EUR\n2022', 
       'Pasivo líquido\nmil EUR\n2022', 'Resultado Explotación\nmil EUR\n2022',  
       'Otros activos líquidos\nmil EUR\n2022','Capital suscrito\nmil EUR\n2022',
       'Otros fondos propios\nmil EUR\n2022','Número empleados\n2022', 'Pasivo fijo\nmil EUR\n2022',
       'Acreedores a L. P.\nmil EUR\n2022',
       'Otros pasivos fijos\nmil EUR\n2022',
       'Deudas financieras\nmil EUR\n2022',
       'Acreedores comerciales\nmil EUR\n2022',
       'Otros pasivos líquidos\nmil EUR\n2022', 'Inmovilizado\nmil EUR\n2022', 'Otros activos fijos\nmil EUR\n2022']]
df_2022_col.head()

Descartamos el df del 2021 al ver una gran cantidad de duplicados y al tener muchos menos datos no repetidos que el resto de los df.

In [ ]:
nif_comunes =  set(df_2020['Código NIF'])&set(df_2022['Código NIF'])
len(nif_comunes)

In [ ]:
paises=list(df_2022_col[['Países exportación']].value_counts().index)
paises

In [ ]:
df_2022_col_comunes =  df_2022_col[df_2022_col['Código NIF'].isin(nif_comunes)]
df_2022_col_comunes.drop_duplicates(inplace=True)
df_2022_col_comunes.reset_index(inplace=True, drop=True)
df_2022_col_comunes.head()

In [ ]:
df_2020_comunes =  df_2020[df_2020['Código NIF'].isin(nif_comunes)]
df_2020_comunes.drop_duplicates(inplace=True)
df_2020_comunes.reset_index(inplace=True, drop=True)
df_2020_comunes.head()

In [ ]:
df_2022_comunes =  df_2022[df_2022['Código NIF'].isin(nif_comunes)]
df_2022_comunes.reset_index(inplace=True, drop=True)
df_2022_comunes.drop_duplicates(inplace=True)
df_2022_comunes.reset_index(inplace=True, drop=True)
df_2022_comunes.head()

In [ ]:
df_2022_col_comunes.columns

In [ ]:
df_2020_comunes.replace('n.d.',np.nan,inplace=True)
df_2022_comunes.replace('n.d.',np.nan,inplace=True)
df_2022_col_comunes.replace('n.d.',np.nan,inplace=True)

In [ ]:
print(df_2022_col_comunes.isna().sum())

In [ ]:
df_2022_col_comunes=df_2022_col_comunes[df_2022_col_comunes['Total activo\nmil EUR\n2022'].notna()]
df_2022_col_comunes.isna().sum()


In [ ]:
df_2020_2022=df_2022_col_comunes.merge(df_2020_ingexp, on='Código NIF', how='left')

### PROCESAMIENTO

In [ ]:
#Factorizar la columna fecha de constitución
df_2020_2022['Fecha constitución'] = pd.to_datetime(df_2020_2022['Fecha constitución'])
df_2020_2022['Empresa reciente'] = df_2020_2022['Fecha constitución'].apply(lambda x: False if x.year < 2019 else True)

In [ ]:
#Factorizar columna
print(df_2020_2022['Provincia'].unique())
df_2020_2022 = pd.get_dummies(df_2020_2022,columns=["Provincia"])


In [ ]:
#Indice
# df_2020_2022.set_index("Código NIF", inplace=True)
# df_2020_2022.index.name=None

In [ ]:
df_2020_2022

# Mapeo de Sectores

In [ ]:
# Función para mapear sectores (1)

def mapeo_sectores(cnae):
    if cnae < 500:
        return "Agricultura"
    elif cnae < 1000:
        return "Extraccion"
    elif cnae < 3500:
        return "Industria"
    elif cnae < 3599:
        return "Energia"
    elif cnae < 4000:
        return "Agua"
    elif cnae < 4400:
        return "Construccion"
    elif cnae < 4800:
        return "Comercio"
    elif cnae < 5500:
        return "Transporte"
    elif cnae < 5800:
        return "Hosteleria"
    elif cnae < 6400:
        return "Comunicaciones"
    elif cnae < 6800:
        return "Financieras"
    elif cnae < 6900:
        return "Inmobiliarias"
    elif cnae < 7700:
        return "Actividades Profesionales"
    elif cnae < 8300:
        return "Actividades Administrativas"
    elif cnae < 8500:
        return "Administracion Publica y Defensa"
    elif cnae < 8600:
        return "Educacion"
    elif cnae < 9000:
        return "Actividades Sanitarias"
    elif cnae < 9400:
        return "Actividades Artisticas"
    else:
        return "Otros"
    

df_2020_2022["Mapeo Sectores"] = df_2020_2022["Código primario CNAE 2009"].apply(mapeo_sectores)

In [ ]:
df_2020_2022["Mapeo Sectores"].value_counts()

In [ ]:
def mapeo_sectores(cnae):
    if cnae < 500:
        return "Otros" # Agricultura
    elif cnae < 1000:
        return "Otros" # Extraccion
    elif cnae < 3500:
        return "Industria"
    elif cnae < 3599:
        return "Otros" # Energia
    elif cnae < 4000:
        return "Otros" # Agua
    elif cnae < 4400:
        return "Construccion e Inmobilarias" # Construccion
    elif cnae < 4800:
        return "Comercio"
    elif cnae < 5500:
        return "Transporte"
    elif cnae < 5800:
        return "Hosteleria"
    elif cnae < 6400:
        return "Otros" # Comunicaciones
    elif cnae < 6800:
        return "Actividades Profesionales y Financieras" # Financieras
    elif cnae < 6900:
        return "Construccion e Inmobilarias" # Inmobiliarias
    elif cnae < 7700:
        return "Actividades Profesionales y Financieras" # Actividades Profesionales
    elif cnae < 8300:
        return "Otros" # Actividades Administrativas
    elif cnae < 8500:
        return "Otros" # Administracion Publica y Defensa
    elif cnae < 8600:
        return "Otros" # Educacion
    elif cnae < 9000:
        return "Otros" # Actividades Sanitarias
    elif cnae < 9400:
        return "Otros" # Actividades Artisticas
    else:
        return "Otros"
    
df_2020_2022["Sectores"] = df_2020_2022["Código primario CNAE 2009"].apply(mapeo_sectores)
df_2020_2022["Sectores"].value_counts()


In [ ]:
df_2020_2022 = pd.get_dummies(df_2020_2022,columns=["Sectores"])

In [ ]:
df_2020_2022 = df_2020_2022.drop(columns=["Sectores_Otros"])

In [ ]:
df_2020_2022.drop(columns=["Código primario CNAE 2009","Literal código CNAE 2009 primario"],inplace=True)

# Comercio en Euros o no

In [ ]:
df_2020_2022["Importador / Exportador"].value_counts()

In [ ]:
list(set(list(set((";".join(list(df_2020_2022["Países exportación"].value_counts().index))).split(";")))+ list(set((";".join(list(df_2020_2022["Países importación"].value_counts().index))).split(";")))))

In [ ]:
no_euro = ["ESLOVAQUIA",
 'ARGENTINA',
 'FILIPINAS',
 'COSTA DE MARFIL',
 'AUSTRALIA',
 'ISRAEL',
 'URUGUAY',
 'CHINA',
 'SUECIA',
 'ORIENTE MEDIO',
 'SUDAMERICA',
 'CANADA',
 'MAURITANIA',
 'NORTE AMERICA',
 'CUBA',
 'DINAMARCA',
 'ARGELIA',
 'NIGERIA',
 'GHANA',
 'VENEZUELA',
 'SUI',
 'ECUADOR',
 'SINGAPUR',
 'NUEVA ZELANDA',
 'GROENLANDIA',
 'ARABIA SAUDI',
 'POLONIA',
 'SUIZA',
 'PAISES ARABES',
 'PAKISTAN',
 'HUNGRIA',
 'BAHREIN',
 'VIETNAM',
 'AFRICA DEL NORTE',
 'MOZAMBIQUE',
 'QATAR',
 'COREA DEL SUR',
 'TAILANDIA',
 'PERU',
 'MALI',
 'MEXICO',
 'PANAMA',
 'MALASIA',
 'UK',
 'RUMANIA',
 'OCEANIA',
 'EMIRATOS ARABES UNIDOS',
 'ASIA',
 'REPUBLICA CHECA',
 'AFRICA',
 'LIBIA',
 'BULGARIA',
 'AMERICA',
 'CHILE',
 'CAMERUN',
 'SERBIA',
 'IRAN',
 'TUNEZ',
 'COSTA RICA',
 'KAZAJSTAN',
 'BRASIL',
 'REPUBLICA DOMINICANA',
 'HONG KONG',
 'SENEGAL',
 'INDONESIA',
 'ANGOLA',
 'CENTRO AMERICA',
 'EGIPTO',
 'MARRUECOS',
 'EEUU',
 'RUSIA',
 'JAPON',
 'COLOMBIA',
 'TURQUIA',
 'UCRANIA',
 'SEYCHELLES',
 'INDIA',
 'SUDAFRICA']

In [ ]:
def pais(lista_paises):
    if type(lista_paises)==float:
        return np.nan
    lista_paises_sp = lista_paises.split(";")
    for pais in no_euro:
        if pais in lista_paises_sp:
            return True
    else:
        return False

exp = df_2020_2022["Países exportación"].apply(pais)
imp = df_2020_2022["Países importación"].apply(pais)

In [ ]:
moneda = []
for i in range(len(exp)):
    if (exp[i] == True) or (imp[i]==True):
        moneda.append(True)
    else:
        moneda.append(False)

In [ ]:
df_2020_2022["Moneda no euro"] = moneda

In [ ]:
df_2020_2022["Moneda no euro"].isna().sum() - df_2020_2022["Importador / Exportador"].value_counts()[0]

In [ ]:
df_2020_2022[["Países importación","Países exportación","Moneda no euro"]].head(20)

### RATIOS

In [ ]:
df_2020_2022['Otros fondos propios\nmil EUR\n2022'] = df_2020_2022['Otros fondos propios\nmil EUR\n2022'].fillna(0)
df_2020_2022['Pasivo Total']=df_2020_2022['Total activo\nmil EUR\n2022']-df_2020_2022['Capital suscrito\nmil EUR\n2022']-df_2020_2022['Otros fondos propios\nmil EUR\n2022']
columnas_sumar=['Pasivo fijo\nmil EUR\n2022', 'Pasivo líquido\nmil EUR\n2022']
df_2020_2022['Pasivo Total'] = df_2020_2022['Pasivo Total'].fillna(df_2020_2022[columnas_sumar].sum(axis=1))
df_2020_2022.loc[df_2020_2022['Pasivo Total'] < 0, 'Pasivo Total'] = 0
df_2020_2022['Endeudamiento']=df_2020_2022['Pasivo Total']/df_2020_2022['Total activo\nmil EUR\n2022']


In [ ]:
columnas_sumar2=['Inmovilizado\nmil EUR\n2022', 'Otros activos fijos\nmil EUR\n2022']
df_2020_2022['Activo circulante\nmil EUR\n2022']=df_2020_2022['Activo circulante\nmil EUR\n2022'].fillna(df_2020_2022['Total activo\nmil EUR\n2022'] - df_2020_2022[columnas_sumar2].sum(axis=1))
df_2020_2022['Pasivo fijo\nmil EUR\n2022']=df_2020_2022['Pasivo fijo\nmil EUR\n2022'].fillna(0)
df_2020_2022['Pasivo líquido\nmil EUR\n2022'] = df_2020_2022['Pasivo líquido\nmil EUR\n2022'].fillna(df_2020_2022['Pasivo Total'] - df_2020_2022['Pasivo fijo\nmil EUR\n2022'])
df_2020_2022['Solvencia']=df_2020_2022['Activo circulante\nmil EUR\n2022']/df_2020_2022['Pasivo líquido\nmil EUR\n2022']
df_2020_2022.loc[df_2020_2022['Solvencia'] < 0, 'Solvencia'] = 0

In [ ]:
df_2020_2022['Evolucion_ventas_total']=df_2020_2022['Ingresos de explotación\nmil EUR\n2022']-df_2020_2022['Ingresos de explotación\nmil EUR\n2020']
df_2020_2022['Evolucion_ventas']=df_2020_2022['Evolucion_ventas_total']/df_2020_2022['Ingresos de explotación\nmil EUR\n2020']
df_2020_2022['Evolucion_ventas'] = df_2020_2022['Evolucion_ventas'].fillna(0)

In [ ]:
df_2020_2022['Resultado del Ejercicio\nmil EUR\n2022'] = df_2020_2022['Resultado del Ejercicio\nmil EUR\n2022'].fillna(0)
df_2020_2022['ROA']=df_2020_2022['Resultado del Ejercicio\nmil EUR\n2022']/df_2020_2022['Total activo\nmil EUR\n2022']

### Categorizar solvencia

In [ ]:
def solv(val):
    if val < 0.7:
        return 0
    elif val < 1:
        return 1
    elif val < 1.5:
        return 2
    else:
        return 3

In [ ]:
df_2020_2022["Solvencia"] = df_2020_2022["Solvencia"].apply(lambda x: solv(x))

In [ ]:
df_2020_2022.columns

In [ ]:
df_procesado=df_2020_2022[["Código NIF", 'Provincia', 'Total activo\nmil EUR\n2022',
       'Empresa reciente', 'Sectores_Actividades Profesionales y Financieras',
       'Sectores_Comercio', 'Sectores_Construccion e Inmobilarias',
       'Sectores_Hosteleria', 'Sectores_Industria',
       'Sectores_Transporte', 'Moneda no euro',
       'Endeudamiento', 'Solvencia', 'Evolucion_ventas', 'ROA']]

In [ ]:
df_procesado.info()

In [ ]:
df_procesado.to_excel('df_procesado.xlsx', index=False)

- ver cuales son las variables que nos interesan y cuales no para eliminarlas, correlacion haciendo dropna?
- ver las filas que tienen casi todo na por si eliminamos las filas y nos quedamos con las variables
- interpolacion en alguna de las columnas?
- pensar que nuevas varibales nos pueden valer.
-pensar los modelos que vamos a utilizar para el clustering, k-means, clustering jerarquico...
- organizar los 4 puntos del trabajo(4 archivos diferentes para cada paso):
1. Iniciar el proyecto requiere primero comprender el significado de las
variables presentes en las bases de datos y realizar una depuración de
estos datos para que sean aptos para trabajar con ellos.

2. Seguidamente, se procederá al cálculo de los ratios financieros.

3. Luego, basándose en los cálculos anteriores, se segmentarán las
empresas mediante el uso de algoritmos de clustering.

4. Tras segmentar las empresas en diferentes grupos, estos se deberán
analizar individualmente para poder seleccionar los productos financieros
que mejor se adecuan a cada tipo de empresa. Se realizará una
investigación y análisis detallado de una amplia gama de productos
financieros disponibles en el mercado. Esto incluirá, pero no se limitará a,
préstamos, líneas de crédito, opciones de inversión, y seguros
empresariales. Esta investigación deberá utilizar como punto de partida al
cliente principal, Inetum, y valorar sus posibilidades a la hora de elegir los
productos financieros.


ESTA ES LA DISTRIBUCIÓN QUE SE PROPONE:
1. Procesamiento
2. Análisis exploratorio
3. Selección del modelo
4. Optimización del modelo
5. Resultados (visualización)

- Cada cosa en un cuaderno diferente
- De cada cuaderno sale un nuevo fichero de datos
- El nuevo fichero de datos se utilizará en el siguiente cuaderno
- Es un proceso secuencial